In [ ]:
from Model.DCVAE import DCVAE
from Model.Utils import PlotHistory, PlotDataAE, Save_Model
from Model.Utils import load_from_tfrecords, convert_to_tfrecords, load_numpy
from keras.optimizers import RMSprop,Adam
import numpy as np
# from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
%matplotlib notebook

# Load DataSet

In [ ]:
load_tfrecords = True 
create_tfrecords = False

path_tfr = 'DataSet/MPS60'
path_npy = '/share_delta/GeoFacies/ModelosDeFacies/ModelMPS60/DataSet/DataSet.npy'
batch_size = 32

In [ ]:
if load_tfrecords:
    if create_tfrecords: 
        x_train, x_test = load_numpy(path_npy,random_state=0,split_data=0.30)
        convert_to_tfrecords(path_tfr, x_train, x_test)
        gen_train, gen_test = load_from_tfrecords(path_tfr, batch_size)
    else:
        try:
            gen_train, gen_test = load_from_tfrecords(path_tfr, batch_size)
        except:
            print("Data not found. Change 'create_tfrecords' to True")
else:
    x_train, x_test = load_numpy(path_npy,random_state=0,split_data=0.30)

# Create Convolutional Variational Autoencoder

In [ ]:
path_weights = os.path.join(path_tfr, 'w60.hdf5')

model = DCVAE(input_shape=(60, 60, 2),filters=[128,64,32],strides=[2,2,1],KernelDim=(3,3,3),hidden_dim=2048,
              latent_dim=500,opt=Adam(lr=0.0001),dropout=0.1,epochs_drop=200, filepath=path_weights)

# Training Network

In [ ]:
num_epochs = 500
if load_tfrecords:
    x_train = gen_train.mps_generator()
    x_val = gen_test.mps_generator()
    model.fit_generator(x_train,
                        num_epochs=num_epochs, verbose=1, 
                        steps_per_epoch = len(gen_train),
                        val_set = x_val,
                        validation_steps = len(gen_test))
else:
    model.fit(x_train, num_epochs=num_epochs, batch_size = batch_size, x_v=x_test,verbose=1) 

# Ploting Train History

In [ ]:
PlotHistory(model.history.history,listKeys=['val_loss','loss']) 

In [ ]:
PlotHistory(model.history.history,listKeys=['val_acc_pred','acc_pred'])

In [ ]:
PlotHistory(model.history.history,listKeys=['lr'])

# Evaluate Model with Test dataset 

In [ ]:
if load_tfrecords:
    x_test = gen_test.get_numpy_batch()

x_rec = model.model.predict(x_test)
PlotDataAE(np.argmax(x_test[:,:,:,:],axis=-1),np.argmax(x_rec[:,:,:,:],axis=-1),digit_size=(60,60))

# Generate random sampling

In [ ]:
x_gen=model.generate(binary=True)
PlotDataAE([],x_gen[:,:,:],digit_size=(60,60),Only_Result=False)